In [2]:
import numpy as np
from simtk import openmm, unit
from simtk.openmm import app
from openmmtools import integrators
from openmmtools.testsystems import WaterBox

import sys
sys.path.append("/Users/rossg/Work/saltswap/saltswap")
from integrators import GHMCIntegrator as GHMC

# Testing energy outputs from GHMC integrator

From version `0.8.2` of `openmmtools`, the GHMC integrator can return the initial and final potential energies for each integration step. This notebook compares the energies returned by the integrator to the `context.getState()` function for 
1. Normal dynamics
2. NCMC Dynamics with parameter perturbation with `setParticleParameters()` and `updateParametersInContext()`

**CONCLUSION:** Energies returned by GHMC integrator match for normal dynamics, but do not agree with `context.getState()` following parameter perturbation.

## Normal dynamics

Using a box of water as the test system, and choosing a timestep that results in high acceptance rates for the GHMC integrator.

In [2]:
size = 20.0*unit.angstrom     # The length of the edges of the water box.
temperature = 300*unit.kelvin
pressure = 1*unit.atmospheres
wbox = WaterBox(box_edge=size,nonbondedMethod=app.PME,cutoff=9*unit.angstrom,ewaldErrorTolerance=1E-6)
ghmc = integrators.GHMCIntegrator(temperature, 1/unit.picosecond, 0.1*unit.femtoseconds)
context = openmm.Context(wbox.system, ghmc)
context.setPositions(wbox.positions)
context.setVelocitiesToTemperature(temperature)
ghmc.step(200)     # thermalisation

accprob = ghmc.getGlobalVariableByName('naccept')/ghmc.getGlobalVariableByName('ntrials')
print 'Acceptance rate for dynamics = {:3f}'.format(accprob) # Should be around 98%

Acceptance rate for dynamics = 0.975000


An example of comparing energies from the integrator and `getState`. They _should_ agree exactly.

In [3]:
naccept = ghmc.getGlobalVariableByName('naccept')
state = context.getState(getEnergy=True)
pot_old = state.getPotentialEnergy()
ghmc.step(1)
if ghmc.getGlobalVariableByName('naccept') > naccept:
    state = context.getState(getEnergy=True)
    pot_new = state.getPotentialEnergy()
    print 'Energy by | State          |   GHMC '
    print '------------------------------------'
    print 'E old     | {:6f}   | {:6f}'.format(pot_old/unit.kilojoule_per_mole, ghmc.getGlobalVariableByName('potential_old'))
    print 'E new     | {:6f}   | {:6f}'.format(pot_new/unit.kilojoule_per_mole, ghmc.getGlobalVariableByName('potential_new'))

Energy by | State          |   GHMC 
------------------------------------
E old     | -5763.435050   | -5763.435050
E new     | -5844.503080   | -5844.503080


It appears they do. Repeating for many iterations:

In [4]:
enew = []
eold = []
ntrials = 500
for trial in range(ntrials):    
    naccept = ghmc.getGlobalVariableByName('naccept')
    state = context.getState(getEnergy=True)
    pot_old = state.getPotentialEnergy()/unit.kilojoule_per_mole
    ghmc.step(1)
    if ghmc.getGlobalVariableByName('naccept') > naccept:
        state = context.getState(getEnergy=True)
        pot_new = state.getPotentialEnergy()/unit.kilojoule_per_mole
        eold.append((pot_old,ghmc.getGlobalVariableByName('potential_old')))
        enew.append((pot_new,ghmc.getGlobalVariableByName('potential_new')))

In [5]:
print 'Fraction of initial energies that match =', len([True for x,y in eold if x==y])*1.0/len(eold)
print 'Fraction of final energies that match =',len([True for x,y in enew if x==y])*1.0/len(enew)

Fraction of initial energies that match = 1.0
Fraction of final energies that match = 0.995926680244


Intial energies consistently match. Slight mismatch with final energies though:

In [6]:
[(x,y) for x,y in enew if x!=y]

[(-8035.988591316374, -8035.990770694392),
 (-8402.965382524693, -8402.963954129315)]

The descrepancy seems to be a matter of imprecision, and of no great concern.

### Updating parameters

Creating the water box as the test system again

In [7]:
size = 20.0*unit.angstrom     # The length of the edges of the water box.
temperature = 300*unit.kelvin
pressure = 1*unit.atmospheres
wbox = WaterBox(box_edge=size,nonbondedMethod=app.PME,cutoff=9*unit.angstrom,ewaldErrorTolerance=1E-6)
ghmc = integrators.GHMCIntegrator(temperature, 1/unit.picosecond, 0.1*unit.femtoseconds)
context = openmm.Context(wbox.system, ghmc)
context.setPositions(wbox.positions)
context.setVelocitiesToTemperature(temperature)
ghmc.step(200)     # thermalisation

accprob = ghmc.getGlobalVariableByName('naccept')/ghmc.getGlobalVariableByName('ntrials')
print 'Acceptance rate = {:3f}'.format(accprob) # Should be around 97%

Acceptance rate = 0.970000


Modifying the non-bonded parameters of the first water molecule to see if that affects the energies that are retured by the GHMC integrator.

In [8]:
force = wbox.system.getForce(2)       # Non-bonded force.
print'Oxygen 1', force.getParticleParameters(0)
print'Hydrogen 1', force.getParticleParameters(1)
print'Hydrogen 2', force.getParticleParameters(2)

Oxygen 1 [Quantity(value=-0.834, unit=elementary charge), Quantity(value=0.3150752406575124, unit=nanometer), Quantity(value=0.635968, unit=kilojoule/mole)]
Hydrogen 1 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
Hydrogen 2 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]


Creating 2 functions to perturb the nonbonded parameters of the 1st water to test the agreement of the energy calculation methods after `updateParametersInContext` in an NCMC protocol.

In [5]:
def switchoff(force,context,frac=0.95):
    """
    Function to modify the non bonded parameters of the non-bonded parameters for the first tip3p water molecules.
    """
    force.setParticleParameters(0,charge=-0.834*frac,sigma=0.3150752406575124*frac,epsilon=0.635968*frac)
    force.setParticleParameters(1,charge=0.417*frac,sigma=0,epsilon=1*frac)
    force.setParticleParameters(2,charge=0.417*frac,sigma=0,epsilon=1*frac)
    force.updateParametersInContext(context)
    
def switchon(force,context):
    """
    Function to reset the non bonded parameters of the non-bonded parameters for the first tip3p water molecules.
    """
    force.setParticleParameters(0,charge=-0.834,sigma=0.3150752406575124,epsilon=0.635968)
    force.setParticleParameters(1,charge=0.417,sigma=0,epsilon=1)
    force.setParticleParameters(2,charge=0.417,sigma=0,epsilon=1)
    force.updateParametersInContext(context)

In [6]:
def NCMC_simple(ntrials):
    """
    Function to compare potential energies calcualted vie 'getState' and the energies returned from the GHMC integrator.
    The non-bonded parameters of the first water molecules will be slowly decoupled. 
    
    
    Just like the NCMC function in SaltSwap, the protocol follows
        propagation -> perturbation -> propagation
    where the first propagation step is outside the loop.    
    
    """
    eold = []
    enew = []
    ghmc.step(1)     # propagation
    naccept = ghmc.getGlobalVariableByName('naccept')
    for n in range(ntrials):
        fraction = 1 - float(n + 1)/ntrials
        # Get the energy BEFORE the parameters are perturbed.
        state = context.getState(getEnergy=True)
        pot_old = state.getPotentialEnergy()/unit.kilojoule_per_mole
        # Saving potential energy via getState and from integrator
        eold.append((pot_old,ghmc.getGlobalVariableByName('potential_new')))
        
        # Perturbation
        switchoff(force,context,frac=fraction)          
        
        # Get the energy AFTER the parameters have been perturbed.
        state = context.getState(getEnergy=True)
        pot_new = state.getPotentialEnergy()/unit.kilojoule_per_mole
        # Propagation. A step is required for GHMC to calculate energy
        ghmc.step(1)     # The 'old' energy for one step should be the energy after perturbation
        enew.append((pot_new,ghmc.getGlobalVariableByName('potential_old')))
    switchon(force,context)         # Reset the non-bonded parameters of the water
    return eold, enew

Running an NCMC perturbation, and seeing the agreement between energies before and after perturbation:

In [11]:
eold, enew = NCMC_simple(100)
print 'Fraction of initial energies that match =', len([True for x,y in eold if x==y])*1.0/len(eold)
print 'Fraction of final energies that match =',len([True for x,y in enew if x==y])*1.0/len(enew)

Fraction of initial energies that match = 0.72
Fraction of final energies that match = 0.29


Printing out the potential energies that **don't** match:

In [12]:
[(x,y) for x,y in eold if x!=y]       # Energies before a perturbation

[(-4231.614510262749, -4231.931195994082),
 (-3806.634617177362, -3807.1440901085007),
 (-3951.8243538296956, -3952.411745872887),
 (-3157.754483972225, -3158.513551353477),
 (-2799.158542065328, -2799.992254527431),
 (-3320.1431229580485, -3321.0253412376915),
 (-3148.8798992618104, -3149.7803721298696),
 (-3931.8398464503553, -3932.7674212759302),
 (-3631.263055251955, -3632.1988150966354),
 (-3780.9080000684917, -3781.8447055953),
 (-3305.654427484027, -3306.592756998405),
 (-3467.6516695901228, -3467.6486614310124),
 (-3622.90366193706, -3623.8367153573927),
 (-3300.1814285161527, -3301.1110140539677),
 (-3460.9492016657314, -3461.873379896555),
 (-2774.3750429078063, -2775.2918016197364),
 (-3122.710302863459, -3123.6175864718243),
 (-2769.231617966594, -2770.133770477245),
 (-2945.089364096464, -2945.98593367837),
 (-2398.6852345952066, -2399.575687743956),
 (-3110.5107034996618, -3111.387735661032),
 (-2935.414410418307, -2936.289349994913),
 (-3435.783847078317, -3436.650736820

In [13]:
[(x,y) for x,y in enew if x!=y]       # Energies after a perturbation

[(-2625.6427968805365, -2623.083928453867),
 (-2810.0074041299376, -2809.609882189863),
 (-2990.3612782289856, -2990.066746834651),
 (-3165.73988051567, -3165.541082356649),
 (-3335.4254415200994, -3335.3163736485294),
 (-3499.2515678565833, -3499.22666621003),
 (-3657.1295513827063, -3657.182881863031),
 (-3809.1499272844667, -3809.2763552050747),
 (-3955.4109167735296, -3955.6057225427503),
 (-4096.1235805537435, -4096.381564060153),
 (-4231.614510262749, -4231.931195994082),
 (-4094.9957466206833, -4095.4161692123453),
 (-3953.533785533087, -3954.000325013636),
 (-3806.634617177362, -3807.1440901085007),
 (-3951.8243538296956, -3952.411745872887),
 (-3804.1559753117763, -3804.8098894827563),
 (-3651.3779635172396, -3652.061821955969),
 (-3492.676100922996, -3493.3864628713636),
 (-3328.1136258449405, -3328.849620053792),
 (-3157.754483972225, -3158.513551353477),
 (-2980.0468766336853, -2980.8642364321713),
 (-2799.158542065328, -2799.992254527431),
 (-2977.3380078757036, -2978.1988

Orginal energies and final energies hardly match. This corroborating what I found in `saltswap`.

## Testing modified NCMC integrator

In [3]:
size = 20.0*unit.angstrom     # The length of the edges of the water box.
temperature = 300*unit.kelvin
pressure = 1*unit.atmospheres
wbox = WaterBox(box_edge=size,nonbondedMethod=app.PME,cutoff=9*unit.angstrom,ewaldErrorTolerance=1E-6)
ghmc = GHMC(temperature, 1/unit.picosecond, 0.1*unit.femtoseconds)    # the modified integrator
context = openmm.Context(wbox.system, ghmc)
context.setPositions(wbox.positions)
context.setVelocitiesToTemperature(temperature)
ghmc.step(200)     # thermalisation

accprob = ghmc.getGlobalVariableByName('naccept')/ghmc.getGlobalVariableByName('ntrials')
print 'Acceptance rate = {:3f}'.format(accprob) # Should be around 97%

Acceptance rate = 0.950000


In [4]:
force = wbox.system.getForce(2)       # Non-bonded force.

In [7]:
eold, enew = NCMC_simple(100)
print 'Fraction of initial energies that match =', len([True for x,y in eold if x==y])*1.0/len(eold)
print 'Fraction of final energies that match =',len([True for x,y in enew if x==y])*1.0/len(enew)

Fraction of initial energies that match = 0.99
Fraction of final energies that match = 0.01


The initial energies agree to a much higher degree. Descrepancy due to imprecision: 

In [8]:
[(x,y) for x,y in eold if x!=y]       # Energies before a perturbation that don't agree

[(-8310.490585109685, -8310.490494971134)]